In [1]:
import os
import sys
import xarray as xr
import pandas as pd
import numpy as np
from datetime import datetime

import matplotlib as mpl
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt

In [2]:
# file_date = '20210821'          #which date to create clean DAWN data CSV for, see "CPEX-AW Well Documented Convection.docx" for time ranges
# start_time0 = '20210821,221000'  #start time to create clean DAWN data CSV for; gauge near-storm environment based on APR plots and IMERG/Hourly DC-8 track on CPEX Portal
# end_time0 = '20210821,234500'    #end time to create clean DAWN data CSV for; gauge near-storm environment based on APR plots and IMERG/Hourly DC-8 track on CPEX Portal

# file_date = '20210822'          #which date to create clean DAWN data CSV for, see "CPEX-AW Well Documented Convection.docx" for time ranges
# start_time0 = '20210822,005000'  #start time to create clean DAWN data CSV for; gauge near-storm environment based on APR plots and IMERG/Hourly DC-8 track on CPEX Portal
# end_time0 = '20210822,020500'    #end time to create clean DAWN data CSV for; gauge near-storm environment based on APR plots and IMERG/Hourly DC-8 track on CPEX Portal

file_date = '20210824'          #which date to create clean DAWN data CSV for, see "CPEX-AW Well Documented Convection.docx" for time ranges
start_time0 = '20210824,175000'  #start time to create clean DAWN data CSV for; gauge near-storm environment based on APR plots and IMERG/Hourly DC-8 track on CPEX Portal
end_time0 = '20210824,202500'    #end time to create clean DAWN data CSV for; gauge near-storm environment based on APR plots and IMERG/Hourly DC-8 track on CPEX Portal

# file_date = '20210826'          #which date to create clean DAWN data CSV for, see "CPEX-AW Well Documented Convection.docx" for time ranges
# start_time0 = '20210826,182000'  #start time to create clean DAWN data CSV for; gauge near-storm environment based on APR plots and IMERG/Hourly DC-8 track on CPEX Portal
# end_time0 = '20210826,222500'    #end time to create clean DAWN data CSV for; gauge near-storm environment based on APR plots and IMERG/Hourly DC-8 track on CPEX Portal

# file_date = '20210828'          #which date to create clean DAWN data CSV for, see "CPEX-AW Well Documented Convection.docx" for time ranges
# start_time0 = '20210828,215000'  #start time to create clean DAWN data CSV for; gauge near-storm environment based on APR plots and IMERG/Hourly DC-8 track on CPEX Portal
# end_time0 = '20210829,000500'    #end time to create clean DAWN data CSV for; gauge near-storm environment based on APR plots and IMERG/Hourly DC-8 track on CPEX Portal

# file_date = '20210901'          #which date to create clean DAWN data CSV for, see "CPEX-AW Well Documented Convection.docx" for time ranges
# start_time0 = '20210901,150000'  #start time to create clean DAWN data CSV for; gauge near-storm environment based on APR plots and IMERG/Hourly DC-8 track on CPEX Portal
# end_time0 = '20210901,180000'    #end time to create clean DAWN data CSV for; gauge near-storm environment based on APR plots and IMERG/Hourly DC-8 track on CPEX Portal

# file_date = '20210904'          #which date to create clean DAWN data CSV for, see "CPEX-AW Well Documented Convection.docx" for time ranges
# start_time0 = '20210904,184000'  #start time to create clean DAWN data CSV for; gauge near-storm environment based on APR plots and IMERG/Hourly DC-8 track on CPEX Portal
# end_time0 = '20210904,200000'    #end time to create clean DAWN data CSV for; gauge near-storm environment based on APR plots and IMERG/Hourly DC-8 track on CPEX Portal


In [3]:
start_time = datetime.strptime(start_time0, '%Y%m%d,%H%M%S')
end_time = datetime.strptime(end_time0, '%Y%m%d,%H%M%S')
day_folder = os.path.join(os.getcwd(), file_date)
dawn_folder = os.path.join(day_folder, 'DAWN_files')
dawn_final_name = os.path.join(day_folder, 'final_dawn_' + file_date + '.csv')

In [4]:
test = xr.open_dataset('/Users/brodenkirch/Desktop/CPEX/Coding/20210821/DAWN_files/CPEXAW-DAWN_DC8_20210821_R0.nc')
test

<xarray.Dataset>
Dimensions:                       (number_profile_records: 904, number_profile_vertical_bins: 305)
Dimensions without coordinates: number_profile_records, number_profile_vertical_bins
Data variables: (12/24)
    AC_Heading                    (number_profile_records) float32 ...
    AC_Pitch                      (number_profile_records) float32 ...
    AC_Roll                       (number_profile_records) float32 ...
    AC_Velocity_North             (number_profile_records) float32 ...
    AC_Velocity_East              (number_profile_records) float32 ...
    AC_Velocity_Down              (number_profile_records) float32 ...
    ...                            ...
    Goodness_of_Fit               (number_profile_vertical_bins, number_profile_records) float32 ...
    Vertical_Integration          (number_profile_vertical_bins, number_profile_records) float32 ...
    Profile_Time                  (number_profile_records) float64 ...
    Profile_Time_Seconds          (number_profile_records) datetime64[ns] ...
    Profile_Date                  (number_profile_records) int32 ...
    QC_flag                       (number_profile_vertical_bins, number_profile_records) int8 ...
Attributes: (12/16)
    creation_date:        2022-04-21T12:26:08.000000Z
    title:                Vertical profiles of horizontal wind and ancillary ...
    summary:              DAWN wind profiles collected from one of the scienc...
    sensor:               Doppler Aerosol Wind Lidar
    platform:             NASA DC-8
    product_version:      1
    ...                   ...
    time_coverage_end:    2021-08-22T02:20:58.000000Z
    geospatial_lat_min:   11.863
    geospatial_lat_max:   20.985
    geospatial_lon_min:   -64.598
    geospatial_lon_max:   -49.077
    gps_ins_data_source:  DC8 MetNav

In [5]:
#loop through each DAWN file to filter/QC the data and add to the given date's final_dawn CSV
ds = xr.open_dataset(os.path.join(dawn_folder, os.listdir(dawn_folder)[0]))  #only 1 DAWN file that contains all the DAWN profiles for the file_date

first_file = True
for a in range(len(ds['Profile_Time_Seconds'])):
    profile = ds.isel(number_profile_records = a)
    prof_time = pd.Timestamp(profile.Profile_Time_Seconds.values)#.strftime("%H%M%S")  #convert numpy.datetime64 to datetime object

    if (prof_time < start_time) or (prof_time > end_time):  #ignore DAWN profiles outside of the convective case's time range
        continue
    else:
        prof_df = profile.to_dataframe()
        
        #QC the data: wind speed <= 50 m/s and >= 0, wind direction <= 360 and >= 0, 
        #             u/v magnitude <= 50 m/s, QC_flag = 0, profile_altitude >= 0
        df_use = prof_df[(prof_df['Wind_Speed'] >= 0) & (prof_df['Wind_Speed'] <= 50) & 
                         (prof_df['Wind_Direction'] >= 0) & (prof_df['Wind_Direction'] <= 360) & 
                         (prof_df['U_comp'].abs() <= 50) & (prof_df['V_comp'].abs() <= 50) & 
                         (prof_df['QC_flag'] == 0) & (prof_df['Profile_Altitude'] >= 0)].copy()
            #^^^ using .copy() to prevent chain-indexing --> https://www.dataquest.io/blog/settingwithcopywarning/

        if len(df_use) != 0:  #i.e., good data exists for the profile
            
            #grab the time of the first good data line (the last index), to be used as the official DAWN time
            dawn_full_time = str(df_use.Profile_Time_Seconds.iloc[0])

            #convert the good, relevant DAWN data to a dataframe and add to the final DAWN CSV file
            df_use = df_use.iloc[::-1]  #reverse the dataframe to order by decreasing height
            dawn_clean_df = pd.DataFrame(columns = ['Time [UTC]','Height [m]', 'Wind Direction [deg]', 'Wind Speed [m/s]', 'U Comp of Wind [m/s]', 'V Comp of Wind [m/s]', 'Latitude [deg]', 'Longitude [deg]'])
            dawn_clean_df['Time [UTC]'] = [dawn_full_time] * len(df_use)  #a list of len(df_use) with the same dawn_full_time value
            dawn_clean_df['Height [m]'] = np.round(list(df_use['Profile_Altitude'] * 1000), 1)
            dawn_clean_df['Wind Direction [deg]'] = np.round(list(df_use['Wind_Direction']), 5)
            dawn_clean_df['Wind Speed [m/s]'] = np.round(list(df_use['Wind_Speed']), 5)
            dawn_clean_df['U Comp of Wind [m/s]'] = np.round(list(df_use['U_comp']), 5)
            dawn_clean_df['V Comp of Wind [m/s]'] = np.round(list(df_use['V_comp']), 5)
            dawn_clean_df['Latitude [deg]'] = np.round(list(df_use['Profile_Latitude']), 7)
            dawn_clean_df['Longitude [deg]'] = np.round(list(df_use['Profile_Longitude']), 7)

            if first_file:
                dawn_clean_df.to_csv(dawn_final_name, index = False)
                first_file = False
            else:
                df_all = pd.read_csv(dawn_final_name)
                df_total = pd.concat([df_all, dawn_clean_df], ignore_index = True)  #concatenates fields with same heading
                df_total.to_csv(dawn_final_name, index = False)


In [6]:
#plot up the available, good DAWN data for the given time range
df_all = pd.read_csv(dawn_final_name)

dawn_fig, dawn_axs = plt.subplots(nrows=1, ncols=2, figsize = (35,20))
dawn_fig.subplots_adjust(wspace=0.3)
dawn_x_ax = pd.to_datetime(df_all['Time [UTC]'])  #convert to datetime objects in order for set_major_formatter(mpl.dates.DateFormatter("%H:%M") to work
dawn_y_ax = df_all['Height [m]']
dawn_axs[0].scatter(dawn_x_ax, dawn_y_ax, s=15, c='k')
dawn_axs[0].tick_params(axis='x', rotation = 50)
dawn_axs[0].tick_params(labelsize=18)
dawn_axs[0].grid(True)
dawn_axs[0].set_xlabel('Time [UTC]', fontsize=30)
dawn_axs[0].set_ylabel('Height [m]', fontsize=30)
dawn_axs[0].set_title('DAWN Availability', fontsize=40)
dawn_axs[0].xaxis.set_major_formatter(mpl.dates.DateFormatter("%H:%M"))
dawn_axs[0].xaxis.set_major_locator(ticker.MaxNLocator(10))
#dawn_axs[0].gcf().set_size_inches(10,13)

#plot up same figure but with wind barbs instead of dots
dawn_u = df_all['U Comp of Wind [m/s]']
dawn_v = df_all['V Comp of Wind [m/s]']
dawn_axs[1].barbs(dawn_x_ax, dawn_y_ax, dawn_u, dawn_v, fill_empty = True, pivot='middle', sizes=dict(emptybarb=0.075), barbcolor = 'k')
#add "np.sqrt(dawn_u**2 + dawn_v**2)" to above line to color code barbs by speed
dawn_axs[1].tick_params(axis='x', rotation = 50)
dawn_axs[1].tick_params(labelsize=18)
dawn_axs[1].grid(True)
dawn_axs[1].set_xlabel('Time [UTC]', fontsize=30)
dawn_axs[1].set_ylabel('Height [m]', fontsize=30)
dawn_axs[1].set_title('DAWN 2-D Wind at Given Times and Heights', fontsize=40)
dawn_axs[1].xaxis.set_major_formatter(mpl.dates.DateFormatter("%H:%M"))
dawn_axs[1].xaxis.set_major_locator(ticker.MaxNLocator(10))
#dawn_axs[1].gcf().set_size_inches(20,25)

dawn_name = os.path.join(day_folder, "DAWN_avail_and_barbs_" + file_date + ".png")
plt.savefig(dawn_name, bbox_inches = 'tight')
plt.clf();

<Figure size 2520x1440 with 0 Axes>

In [7]:
#Calculate deep layer (500m - 7622.5m) shear for each qualifying DAWN profile using SHARPpy direct shear calculation method
#Results recorded in DAWN_Shear_calculations.csv

#this step is done using DAWN_shear.py, so execute and see that script for how it's done
